# Notebook for classsifying using LSTM and Neural  Netwrok.


# IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import ngrams
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import collections
from wordcloud import WordCloud
import re

import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D,SpatialDropout1D
from sklearn.model_selection import train_test_split 
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score 
from keras.losses import BinaryCrossentropy
from datetime import datetime, timedelta
from glob import glob
import itertools
import nltk

from bs4 import BeautifulSoup
import unicodedata
import re

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist

import warnings
warnings.filterwarnings("ignore")
import tensorflow


#  Loading Data

In [ ]:
status_paths = glob("../data/statuses/*.csv")
len(status_paths)

In [ ]:
def parse_status_csv(path):
    return pd.read_csv(
        path,
        parse_dates=['status_published'],
        date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'),
    ).assign(page_id=lambda x: x["status_id"].str.split("_").str[0])

In [ ]:
_statuses = pd.concat([ parse_status_csv(path) for path in status_paths ])\
    .drop_duplicates(subset=["status_id"])

In [ ]:
sites = pd.read_csv('../data/all-partisan-sites.csv')
domain = pd.read_csv('../data/domaintools-whois-results.csv')
page_info = pd.read_csv('../data/pages-info.csv')

In [ ]:
# Checking the size of the datasets
print('The shape of the site data set is :',sites.shape)
print('The shape of the status data set is :',_statuses.shape)
print('The shape of the domain data set is :',domain.shape)
print('The shape of the page_info data set is :',page_info.shape)

In [ ]:
_statuses.head()

In [ ]:
sites.head()

In [ ]:
page_info.head()

In [ ]:
domain.head()

In [ ]:
def unique_values(df):
    name =[x for x in globals() if globals()[x] is df][0]
    print("UNIQUE VALUES  OF DATAFRAME %s" % name +' ARE:')
    for col in df.columns:
        print(" {column}: {length} ".format(column =col, length = len(df[col].unique())))
    print('\n')


In [ ]:
def column_type(df):
    name =[x for x in globals() if globals()[x] is df][0]
    print("THE DATA TYPES OF COLUMNS FOR DATAFRAME %s" % name +' ARE:')
    for col in df.columns:
        print(" {column}: {length} ".format(column =col, length = df[col].dtypes))
    print('\n')

In [ ]:
for dataset in [_statuses,domain,sites,page_info]:
    unique_values(dataset)

In [ ]:
for dataset in [_statuses,domain,sites,page_info]:
    column_type(dataset)

In [ ]:
# Getting some statistic information about political_category 
print(sites['political_category'].describe()) 

# Pre-processing

In [ ]:
def checking_nan(df):
    missing_values=df.isnull().sum().sort_values(ascending=False)
    percent_nan = (df.isnull().sum() / df.isnull().count()*100).sort_values(ascending=False)
    nan_values = pd.concat([missing_values, percent_nan], axis=1, keys=['missing_values', 'Percent'])
    nan_values=pd.DataFrame(nan_values)
    return nan_values

In [ ]:
def droping_nan(df,thre):
    # Dropping columns that having more than thre% of the missing values
    threshold=int(len(df)*thre)
    df=df.dropna(axis=1,thresh=threshold)
    return df

In [ ]:
def checking_nan_after_removing(df):
    # After removing the columns that has more than 70% null values, we check again which columns has NULL values
    missing_values=df.isnull().sum().sort_values(ascending=False)
    missing_values=pd.DataFrame(df.isnull().sum().sort_values(ascending=False),columns=['missing_values'])
    miss_val=missing_values[missing_values['missing_values']>0]
    return miss_val

## Cheking nan_value

In [ ]:
sites_nan=checking_nan(sites) # checking nan_values for sites dataset
sites_nan

In [ ]:
page_nan=checking_nan(page_info) # checking nan_values for page_info dataset
page_nan

In [ ]:
_status_nan=checking_nan(_statuses) # checking nan_values for statuses dataset
_status_nan

In [ ]:
domain_nan=checking_nan(domain) # checking nan_values for domain_ dataset
domain_nan

## Dropping all column more than a particular percent of nan_values

In [ ]:
sites=droping_nan(sites,0.7)  # dropping all columns having more than 85% nan_values for sites data set
sites.head()

In [ ]:
page_info=droping_nan(page_info,0.7)  # dropping all columns having more than 85% nan_values page data set
page_info.head()

In [ ]:
domain=droping_nan(domain,0.7)  # dropping all columns having more than 85% nan_values domain data set
domain.head()

## Cheking how many NAN-values we still have after removing some

In [ ]:
sites_after_rm_nan=checking_nan_after_removing(sites) # Checking if there is still any nan values in dataframe
sites_after_rm_nan

In [ ]:
#Dispalying sites dataframe with its corresponding nan 
sites_nan_rows=sites[sites.isna().any(axis=1)]
sites_nan_rows


In [ ]:
page_after_rm_nan=checking_nan_after_removing(page_info) 
page_after_rm_nan

In [ ]:
# Dispalying page_info dataframe with its corresponding nan 
page_nan_rows=page_info[page_info.isna().any(axis=1)]
page_nan_rows.head()


In [ ]:
domain_after_rm_nan=checking_nan_after_removing(domain) 
domain_after_rm_nan

In [ ]:
def remove_url(text):
    text = str(text).replace('www.', '').replace('http://', '')
    return text

In [ ]:
page_info['website'] = page_info['website'].apply(remove_url).str.lower() # removing http://www.from page_info website


In [ ]:
page_info["page_id"] = page_info["page_id"].astype(int).astype(str) # Converting page_id type from int64 to object to get datasets merged


In [ ]:
merged_site_page=pd.merge(sites, page_info, left_on="fb_id", right_on="page_id") # Combining site and page dataset based on Fb_id

In [ ]:
merged_status_sit_page=pd.merge(_statuses, merged_site_page, left_on="page_id", right_on="page_id") # combining merged_site_page and statuses  

In [ ]:
merged_data=merged_status_sit_page.join( merged_status_sit_page['status_id'].str.split('_', expand=True).rename(columns={0:'fb_ID', 1:'status_ID'}))
merged_data.head()

In [ ]:
merged_data.isnull().sum() # Checking if there is any NAN-values in merged data set

In [ ]:
merged_data.drop(['fb_ID', 'link_name','page_id','site','about','status_id','status_link'], axis = 1,inplace=True) # removing unnecessary columns
merged_data.rename(columns = {'status_ID':'status_id'}, inplace = True)


In [ ]:
cols_to_move = ['status_id', 'fb_id','status_message','political_category','macedonian','website']
merged_data= merged_data[ cols_to_move + [ col for col in merged_data.columns if col not in cols_to_move ] ]

In [ ]:
merged_data.head()

In [ ]:
merged_data.isnull().sum()

In [ ]:
merged_data.dropna(subset=['status_message'],axis=0,inplace=True)  # Removing null values in 'about' columns

In [ ]:
final_data=merged_data

In [ ]:
def text_cleaning(text):
    word= BeautifulSoup(text, 'html.parser').get_text()
    word= re.sub(r'https?://\S+|www\.\S+', '', word)
    word= unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    word=re.sub(r'[^a-zA-Z]', ' ', word)
    word=re.sub(r'^\s*|\s\s*', ' ', word).strip()
    word=''.join(ch for ch, _ in itertools.groupby(word))
    word = (re.compile(r'\b(' + r'|'.join(stopwords.words("english")) + r')\b\s*')).sub('', word).lower()
    word=(re.sub('([A-Z])', r' \1', word)).replace('  ',' ')

    return word

In [ ]:
final_data['status_message']=final_data['status_message'].apply(text_cleaning) # Cleaning the text data

In [ ]:
# We need numerical values in ML, therefore we convert political_category variable to a numeric variable
final_data['political_category'] = final_data['political_category'].map({'right':1 ,'left':0})  
  
final_data["fb_id"] = final_data["fb_id"].astype(str).astype(int) # Converting fb_column from object type to int64


In [ ]:
final_data.head()

# Visualization

In [ ]:
final_data['political_category'].value_counts()

In [ ]:
final_data['macedonian'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='macedonian',data=final_data)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='political_category',data=final_data)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
status_messages = ' '.join(map(str,page_info['about']))

wordcloud=WordCloud(width=1920,height=1080).generate(status_messages)
fig=plt.figure(figsize=(10,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Word2vec preproccesing using CBOW and Tokenizer

In [ ]:
def X_tokenizer_vocab(x, max_length):
   
    tokenizer= Tokenizer()  # This class allows to vectorize a text corpus, by turning each text into a sequence of integers 
    tokenizer.fit_on_texts(x) #  A list of sequence. A "sequence" is a list of integer word indices.
    X = tokenizer.texts_to_sequences(x)  # By this code we get a list of sequence I.e. for each particular word a sequence number is assigned. 
    X= pad_sequences(X, maxlen=max_length) #  every sequence are of max_length
    vocab_size=len(tokenizer.word_index)+1  # The length of vocabularies in the text data. This is the input of LSTM model and column size of Embedding matrix.
    vocab= tokenizer.word_index # Vocabularies in text data used in Embedding matrix.

    return X,vocab_size,vocab

In [ ]:
# function for creating Embedding Matrix
def get_weight_matrix(model,weight_matrix_rows,weight_matrix_rows_column,vocabulary):
    weight_matrix= np.zeros((weight_matrix_rows,weight_matrix_rows_column))
    
    for word,i in vocabulary.items():
        weight_matrix[i] = model.wv[word]
    return weight_matrix

In [ ]:
df=final_data.copy()

In [ ]:
x= [d.split() for d in df['status_message'].tolist()] # We get our text data in the form list of words
x= [i for i in x if i != []]

In [ ]:
X,vocab_size, vocab=X_tokenizer_vocab(x, max_length=40) 

In [ ]:
len(X)

In [ ]:
y=df['political_category'].values
y=y[:len(X)]
len(y)

In [ ]:
DIM=100
max_length=40
w2v_model_cbow=gensim.models.Word2Vec(sentences=x,vector_size=DIM,window=10,min_count=1,sg=0) # Creating Word2vect model

In [ ]:
len(w2v_model_cbow.wv) # lengh of created unique word

In [ ]:
w2v_model_cbow.wv.key_to_index 

In [ ]:
w2v_model_cbow.wv # By executing this model our text data is  converteted to a set of vector 

In [ ]:
w2v_model_cbow.wv['us']

In [ ]:
w2v_model_cbow.wv.most_similar('us') # Most similiar word for a specific letter

In [ ]:
embedding_vectors= get_weight_matrix(w2v_model_cbow,vocab_size,DIM,vocab)

In [ ]:
embedding_vectors.shape # rows are the lenght of created unique word and columns is size of  dim

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y) # Using SKit-learn libirary to split our data into training data and test data


In [ ]:
print('The shape of the X_train is :',X_train.shape)
print('The shape of the X_test is :',X_test.shape)
print('The shape of the y_train is :',y_train.shape)
print('The shape of the y_test is :',y_test.shape)

# Creating ML model

In [ ]:
def model(classifier=''):
    model= Sequential()

    if classifier=='LSTM':
        model= Sequential()
        model.add(Embedding(vocab_size,output_dim=DIM,weights=[embedding_vectors],input_length=max_length,trainable=False))
        model.add(LSTM(units=128))
        model.add(Dense(32,activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
    else:
        model.add(Dense(30,activation='relu',input_dim=X_train.shape[1]))
        model.add(Dense(60,activation='sigmoid'))
        model.add(Dense(60,activation='relu'))
        model.add(Dense(20,activation='tanh'))
        model.add(Dense(70,activation='tanh'))
        model.add(Dense(1,activation='sigmoid'))
        model.compile(optimizer= tensorflow.keras.optimizers.Adam(learning_rate=0.001),loss=BinaryCrossentropy(from_logits=False),metrics=['acc'])

    return model



## ML model uisng LSTM

In [ ]:
LSTM_model=model(classifier='LSTM') # Model created using LSTM

In [ ]:
LSTM_model.summary()

In [ ]:
LSTM_model.fit(X_train,y_train,validation_split=0.3,epochs=4)

In [ ]:
y_train_pred_lstm= (LSTM_model.predict(X_train)>=0.5).astype(int)
print( 'LSTM got the following accuracy on the train set',accuracy_score(y_train,y_train_pred_lstm))


In [ ]:
y_test_pred_lstm= (LSTM_model.predict(X_test)>=0.5).astype(int)
print( 'LSTM got the following accuracy on the test set',accuracy_score(y_test,y_test_pred_lstm))


## Neural Netwok 

In [ ]:
NN_model=model(classifier='NN')

In [ ]:
NN_model.summary()

In [ ]:
NN_model.fit(X_train,y_train,validation_split=0.3,epochs=5)


In [ ]:
y_train_pred_NN= (NN_model.predict(X_train)>=0.5).astype(int)
print( 'NN got the following accuracy on the train set',accuracy_score(y_train,y_train_pred_NN))


In [ ]:
y_test_pred_NN= (NN_model.predict(X_test)>=0.5).astype(int)
print( 'NN got the following accuracy on the test set',accuracy_score(y_test,y_test_pred_NN))
